In [1]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
from omnibus.data import plot, Field
from omnibus.formats.output import load
import omnibus.converters.meshtal as meshtal
import omnibus.converters.mctal as mctal
from cycler import cycler

/home/krowland/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#plot_dir = '../img/dlvn-plots/'
plot_dir = '../img/dlvn-plots-mc-vr/'
fwc_base_path = '/home/krowland/ldo-results/dlvn-fwcadis/'
cad_base_path = '/home/krowland/ldo-results/dlvn-cadis/'
fwd_path = '/fwd_solution/denovo-forward.out.h5'
fwc_adj_path = '/fwcadis_adj_solution/denovo-adjoint.out.h5'
cad_adj_path = '/adj_solution/denovo-adjoint.out.h5'
mctal_path = '/output/mctal'
meshtal_path = '/output/meshtal'

In [3]:
plot_types = ["fwd", "fwc-adj","cad-adj", "mcnp"]
for ptype in plot_types:
    if not os.path.exists(plot_dir+"/"+ptype):
        os.makedirs(plot_dir+"/"+ptype)

In [4]:
results = []
for quad_type in next(os.walk(fwc_base_path))[1]:
    for run in next(os.walk(fwc_base_path+quad_type))[1]:
        results.append({"type":quad_type,
                        "name":run,
                        "order": int(run[-2:])})
        
for run in results:
    if run["type"] == "ldo":
        run["angles"] = (run["order"]+1)*(run["order"]+1)
        run["label"]  = "LDO"
    elif run["type"] == "qr":
        run["angles"] = (run["order"])*(run["order"])*8
        run["label"]  = "QR"
    elif run["type"] == "galerkin":
        run["angles"] = run["order"]*(run["order"]+2)
        run["label"]  = "Galerkin"
    elif run["type"] == "ldfe":
        run["angles"] = 8*np.power(4,(run["order"]+1))
        run["label"]  = "LDFE"
    else:
        print "No angle formula for quadrature type " + run["type"]

In [5]:
def plot_outlines():
    # top void block
    plt.plot([0,16*2.54], [40*2.54,40*2.54], 'k-')
    plt.plot([0,16*2.54], [44*2.54,44*2.54], 'k-')
    plt.plot([16*2.54,16*2.54], [40*2.54,44*2.54], 'k-')
    # bottom void block
    plt.plot([0,8*2.54], [8*2.54,8*2.54], 'k-')
    plt.plot([0,8*2.54], [12*2.54,12*2.54], 'k-')
    plt.plot([8*2.54,8*2.54], [8*2.54,12*2.54], 'k-')
    # inner poly block
    plt.plot([8*2.54,8*2.54], [12*2.54,32*2.54], 'k-')
    plt.plot([0*2.54,8*2.54], [32*2.54,32*2.54], 'k-')
    # some misc boundaries to do the rest
    plt.plot([8*2.54,16*2.54], [8*2.54,8*2.54], 'k-')
    plt.plot([16*2.54,16*2.54], [8*2.54,36*2.54], 'k-')
    plt.plot([14*2.54,16*2.54], [36*2.54,36*2.54], 'k-')
    plt.plot([14*2.54,14*2.54], [24*2.54,40*2.54], 'k-')
    plt.plot([12*2.54,12*2.54], [8*2.54,24*2.54], 'k-')
    plt.plot([12*2.54,14*2.54], [24*2.54,24*2.54], 'k-')
    plt.plot([14*2.54,14*2.54], [8*2.54,22*2.54], 'k-')
    plt.plot([14*2.54,16*2.54], [22*2.54,22*2.54], 'k-')
    # detector 14 tally (#14)
    plt.plot([12*2.54,14*2.54], [37*2.54,37*2.54], 'k-')
    plt.plot([12*2.54,14*2.54], [39*2.54,39*2.54], 'k-')
    plt.plot([12*2.54,12*2.54], [37*2.54,39*2.54], 'k-')
    # detector 24 tally (#12)
    plt.plot([9*2.54,11*2.54],  [37*2.54,37*2.54], 'k-')
    plt.plot([9*2.54,11*2.54],  [39*2.54,39*2.54], 'k-')
    plt.plot([9*2.54,9*2.54],   [37*2.54,39*2.54], 'k-')
    plt.plot([11*2.54,11*2.54], [37*2.54,39*2.54], 'k-')
    # detector 34 tally (#13)
    plt.plot([10*2.54,12*2.54], [29*2.54,29*2.54], 'k-')
    plt.plot([10*2.54,12*2.54], [31*2.54,31*2.54], 'k-')
    plt.plot([10*2.54,10*2.54], [29*2.54,31*2.54], 'k-')
    plt.plot([12*2.54,12*2.54], [29*2.54,31*2.54], 'k-')
    # detector 44 tally (#11)
    plt.plot([9*2.54,11*2.54],  [21*2.54,21*2.54], 'k-')
    plt.plot([9*2.54,11*2.54],  [23*2.54,23*2.54], 'k-')
    plt.plot([9*2.54,9*2.54],   [21*2.54,23*2.54], 'k-')
    plt.plot([11*2.54,11*2.54], [21*2.54,23*2.54], 'k-')
    # detector 54 tally (#9)
    plt.plot([5*2.54,7*2.54], [37*2.54,37*2.54], 'k-')
    plt.plot([5*2.54,7*2.54], [39*2.54,39*2.54], 'k-')
    plt.plot([5*2.54,5*2.54], [37*2.54,39*2.54], 'k-')
    plt.plot([7*2.54,7*2.54], [37*2.54,39*2.54], 'k-')
    # detector 64 tally (#5)
    plt.plot([0*2.54,1*2.54], [37*2.54,37*2.54], 'k-')
    plt.plot([0*2.54,1*2.54], [39*2.54,39*2.54], 'k-')
    plt.plot([0*2.54,0*2.54], [37*2.54,39*2.54], 'k-')
    plt.plot([1*2.54,1*2.54], [37*2.54,39*2.54], 'k-')

forward results plotting

In [6]:
for run in results:
    fullpath = fwc_base_path+run["type"]+"/"+run["name"]+fwd_path
    dnv = load(fullpath)['denovo'].extract()
    axes = dnv.flux.axes[1:]
    flux_data = np.sum(dnv.flux.data, axis=0)
    flux = Field(name='flux', axes=axes, data=flux_data)
    run["fwd_flux"] = flux
    run["flux_tally"] = 0.0
    status = fwc_base_path+run["type"]+"/"+run["name"]+'/output/status.log'
    f = open(status)
    for line in f.readlines():
        if "finished executing Denovo" in line:
            timeline = line
            break
    f.close()
    timeline = timeline.split()[-2]
    run["time"] = float(timeline)/60.0
    plots = plot(flux.xs(y=68.58), norm=LogNorm(vmin=1e-10, vmax=1e-1))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    title_name = run["label"]+" "+"%02d" % (run["order"],)
    flux_str = ' $\phi\ \\left[\\frac{n}{cm^2\cdot s}\\right]$'
    ax.set_title(title_name + flux_str + ' at $y = 68.58$ cm\n')
    plot_outlines()
    plt.rcParams.update({'axes.titlesize': 'medium'})
    plot_name = 'fwd/flux-'+str(run["name"])+'-slice.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
    if run["type"] == "ldo" and run["order"] == 11: # 144 angles
        ldo_dict = run
    if run["type"] == "qr" and run["order"] == 4: # 128 angles
        qr_dict = run
    if run["type"] == "galerkin" and run["order"] == 4: # P5 expansion
        gkn_dict = run
    if run["type"] == "ldfe" and run ["order"] == 1: # 128 angles
        ldfe_dict = run

Loading HDF5 file...
INFO: Loaded Omnibus output data from '/home/krowland/ldo-results/dlvn-fwcadis/galerkin/gkn04/fwd_solution/denovo-forward.out.h5':
      'Untitled' calculated on 2018FEB23 17:51
      using Exnihilo version 6.2 (branch 'ldo' #8f4b859b on 2018JAN27), Scale version 6.3 (r23620: #167cc003 on 2017OCT13)
            ...finished loading HDF5 file
Loading HDF5 file...
INFO: Loaded Omnibus output data from '/home/krowland/ldo-results/dlvn-fwcadis/galerkin/gkn08/fwd_solution/denovo-forward.out.h5':
      'Untitled' calculated on 2018FEB23 18:24
      using Exnihilo version 6.2 (branch 'ldo' #8f4b859b on 2018JAN27), Scale version 6.3 (r23620: #167cc003 on 2017OCT13)
            ...finished loading HDF5 file
Loading HDF5 file...
INFO: Loaded Omnibus output data from '/home/krowland/ldo-results/dlvn-fwcadis/galerkin/gkn06/fwd_solution/denovo-forward.out.h5':
      'Untitled' calculated on 2018FEB23 17:55
      using Exnihilo version 6.2 (branch 'ldo' #8f4b859b on 2018JAN27), S

forward results plotting for paper

In [ ]:
runs = [ldo_dict, qr_dict, gkn_dict, ldfe_dict]
paper_plot_dir = '/home/krowland/ldo-deterministic/'

for run in runs:
    plots = plot(run["fwd_flux"].xs(y=68.58), norm=LogNorm(vmin=1e-10, vmax=1e-1))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    title_name = run["label"]
    flux_str = ' $\phi\ \\left[\\frac{n}{cm^2\cdot s}\\right]$'
    ax.set_title(title_name + flux_str + ' at $y = 68.58$ cm\n')
    plot_outlines()
    plt.rcParams.update({'axes.titlesize': 'medium'})
    plot_name = 'dlvn-fwd-flux-'+str(run["name"])+'.eps'
    plt.savefig(paper_plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())

fwd diff plotting

In [ ]:
ldo_flux = ldo_dict["fwd_flux"]
std_quads = [qr_dict,gkn_dict,ldfe_dict]

f = open(plot_dir+'fwd-diff.txt','w')
for quad in std_quads:
    std_flux = quad["fwd_flux"]
    abs_diff = np.abs(ldo_flux - std_flux)
    rel_diff = abs_diff/std_flux
    aflux_diff = Field(name='abs_diff', axes=ldo_flux.axes, data=abs_diff)
    rflux_diff = Field(name='rel_diff', axes=ldo_flux.axes, data=rel_diff)
    plots = plot(aflux_diff.xs(y=68.58), norm=LogNorm(vmin=1e-6, vmax=1e1))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    ax.set_title('LDO/'+quad["label"]+' abs flux diff at $y = 68.58$ cm\n')
    plot_outlines()
    plot_name = 'fwd/flux-diff-abs-'+quad["name"]+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
    #
    plots = plot(rflux_diff.xs(y=68.58),norm=LogNorm(vmin=1e-6, vmax=1e1))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    ax.set_title('LDO/'+quad["label"]+' $\phi_{\mathrm{diff}}$ at $y = 68.58$ cm\n')
    plot_outlines()
    plt.rcParams.update({'axes.titlesize': 'medium'})
    plot_name = 'fwd/flux-diff-rel-'+quad["name"]+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
    #
    print 'LDO/'+quad["label"]
    num_data = map(float,rflux_diff.xs(y=68.58).data.ravel())
    print "max diff " + str(max(num_data))
    print "min diff " + str(min(num_data))
    print "avg diff " + str(np.mean(num_data))
    f.write('LDO/'+quad["label"]+'\n')
    f.write("min diff " + str(min(num_data))+'\n')
    f.write("max diff " + str(max(num_data))+'\n')
    f.write("avg diff " + str(np.mean(num_data))+'\n')

std_quads = [gkn_dict,ldfe_dict]
for quad in std_quads:
    rel_diff = np.abs(qr_dict["fwd_flux"]-quad["fwd_flux"])/qr_dict["fwd_flux"]
    rflux_diff = Field(name='rel_diff', axes=ldo_flux.axes, data=rel_diff)
    num_data = map(float,rflux_diff.xs(y=68.58).data.ravel())
    print quad["label"]+"/"+qr_dict["label"]
    print "max diff " + str(max(num_data))
    print "min diff " + str(min(num_data))
    print "avg diff " + str(np.mean(num_data))
    f.write(quad["label"]+"/"+qr_dict["label"]+'\n')
    f.write("min diff " + str(min(num_data))+'\n')
    f.write("max diff " + str(max(num_data))+'\n')
    f.write("avg diff " + str(np.mean(num_data))+'\n')
    
f.close()

fwd comparison with benchmark data

In [ ]:
det5  = {'pos': 5, 'exp_flux': 6.97e-8, 
         'x_min': -2.54, 'x_max':  2.54, # [-1,1] inches
         'y_min': 66.04, 'y_max': 71.12,
         'z_min': 93.98, 'z_max': 99.06} # [37,39] inches
det9  = {'pos': 9, 'exp_flux': 1.57e-7, 
         'x_min': 12.7,  'x_max': 17.78, # [5,7] inches
         'y_min': 66.04, 'y_max': 71.12,
         'z_min': 93.98, 'z_max': 99.06} # [37,39] inches
det11 = {'pos': 11, 'exp_flux': 8.81e-6, 
         'x_min': 22.86, 'x_max': 27.94, # [9,11] inches
         'y_min': 66.04, 'y_max': 71.12,
         'z_min': 53.34, 'z_max': 58.42} # [21, 23] inches
det12 = {'pos': 12, 'exp_flux': 2.6e-7, 
         'x_min': 22.86, 'x_max': 27.94, # [9,11] inches
         'y_min': 66.04, 'y_max': 71.12,
         'z_min': 93.98, 'z_max': 99.06} # [37,39] inches
det13 = {'pos': 13, 'exp_flux': 1.42e-6, 
         'x_min': 25.4, 'x_max': 30.48, # [10,12] inches
         'y_min': 66.04, 'y_max': 71.12,
         'z_min': 73.66, 'z_max': 78.74} # [29,31] inches
det14 = {'pos': 14, 'exp_flux': 2.74e-7, 
         'x_min': 30.48, 'x_max': 35.56, # [12,14] inches
         'y_min': 66.04, 'y_max': 71.12,
         'z_min': 93.98, 'z_max': 99.06} # [37,39] inches

dets = [det5,det9,det11,det12,det13,det14]
quads = [qr_dict,gkn_dict,ldfe_dict,ldo_dict]

# get spatial mesh, doesn't matter which run since they're the same
x_mesh = dnv.mesh.x
y_mesh = dnv.mesh.y
z_mesh = dnv.mesh.z

for det in dets:
    x_vals = x_mesh[np.where(np.logical_and(x_mesh>=det['x_min'], x_mesh<=det['x_max']))]
    det['x_centers'] = (x_vals[1:] + x_vals[:-1]) / 2
    y_vals = y_mesh[np.where(np.logical_and(y_mesh>=det['y_min'], y_mesh<=det['y_max']))]
    det['y_centers'] = (y_vals[1:] + y_vals[:-1]) / 2
    z_vals = z_mesh[np.where(np.logical_and(z_mesh>=det['z_min'], z_mesh<=det['z_max']))]
    det['z_centers'] = (z_vals[1:] + z_vals[:-1]) / 2

for quad in quads:
    for det in dets:
        det[quad["label"]] = 0.0
        for z in det['z_centers']:
            for y in det['y_centers']:
                for x in det['x_centers']:
                    det[quad['label']] += quad["fwd_flux"].xs(x=x,y=y,z=z).data
        det[quad['label']+'_diff'] = (np.abs(det[quad['label']]-det['exp_flux'])/det['exp_flux'])*100

# all deterministic results outside of experimental error
# so, let's look at the percent difference for each case
f = open(plot_dir+'fwd-det.txt','w')
for det in dets:
    print "Detector  " + str(det['pos'])
    print "Exp. flux " + str(det['exp_flux'])
    print "QR flux   " + str(det['QR']) + \
          " % diff "   + str(det['QR_diff'])
    print "Gkn flux  " + str(det['Galerkin']) + \
          " % diff "   + str(det['Galerkin_diff'])
    print "LDFE flux " + str(det['LDFE']) + \
          " % diff "   + str(det['LDFE_diff'])
    print "LDO flux  " + str(det['LDO']) + \
          " % diff "   + str(det['LDO_diff'])
    f.write("Detector  " + str(det['pos'])+'\n')
    f.write("Exp. flux " + str(det['exp_flux'])+'\n')
    f.write("QR flux   " + str(det['QR']) + \
          " % diff "   + str(det['QR_diff'])+'\n')
    f.write("Gkn flux  " + str(det['Galerkin']) + \
          " % diff "   + str(det['Galerkin_diff'])+'\n')
    f.write("LDFE flux " + str(det['LDFE']) + \
          " % diff "   + str(det['LDFE_diff'])+'\n')
    f.write("LDO flux  " + str(det['LDO']) + \
          " % diff "   + str(det['LDO_diff'])+'\n')
f.close()

CADIS adjoint plotting

In [ ]:
for run in results:
    fullpath = cad_base_path+run["type"]+"/"+run["name"]+cad_adj_path
    dnv = load(fullpath)['denovo'].extract()
    axes = dnv.flux.axes[1:]
    flux_data = np.sum(dnv.flux.data, axis=0)
    flux = Field(name='flux', axes=axes, data=flux_data)
    run["cad_adj_flux"] = flux
    plots = plot(flux.xs(y=68.58), norm=LogNorm(vmin=1e-6, vmax=1e2))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    title_name = run["label"]+" "+"%02d" % (run["order"],)
    flux_str = ' CADIS $\phi^{\dagger}\ \\left[\\frac{n}{cm^2\cdot s}\\right]$'
    ax.set_title(title_name + flux_str + ' at $y = 68.58$ cm\n')
    plot_outlines()
    plt.rcParams.update({'axes.titlesize': 'medium'})
    plot_name = 'cad-adj/flux-'+str(run["name"])+'-slice.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())

CADIS adjoint diff plotting

In [ ]:
ldo_flux = ldo_dict["cad_adj_flux"]
std_quads = [qr_dict,gkn_dict,ldfe_dict]

f = open(plot_dir+'cad-adj-diff.txt','w')
for quad in std_quads:
    std_flux = quad["cad_adj_flux"]
    abs_diff = np.abs(ldo_flux - std_flux)
    rel_diff = abs_diff/std_flux
    aflux_diff = Field(name='abs_diff', axes=ldo_flux.axes, data=abs_diff)
    rflux_diff = Field(name='rel_diff', axes=ldo_flux.axes, data=rel_diff)
    plots = plot(aflux_diff.xs(y=68.58), norm=LogNorm(vmin=1e-5, vmax=1e2))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    ax.set_title('LDO/'+quad["label"]+' abs flux diff at $y = 68.58$ cm\n')
    plot_outlines()
    plot_name = 'cad-adj/flux-diff-abs-'+quad["name"]+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
    #
    plots = plot(rflux_diff.xs(y=68.58),norm=LogNorm(vmin=1e-5, vmax=1e2))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    ax.set_title('LDO/'+quad["label"]+' $\phi^{\dagger}_{\mathrm{diff}}$ at $y = 68.58$ cm\n')
    plot_outlines()
    plt.rcParams.update({'axes.titlesize': 'medium'})
    plot_name = 'cad-adj/flux-diff-rel-'+quad["name"]+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
    #
    print 'LDO/'+quad["label"]
    num_data = map(float,rflux_diff.xs(y=68.58).data.ravel())
    print "max diff " + str(max(num_data))
    print "min diff " + str(min(num_data))
    print "avg diff " + str(np.mean(num_data))
    f.write('LDO/'+quad["label"]+'\n')
    f.write(str(min(num_data))+' & ')
    f.write(str(round(max(num_data),5))+' & ')
    f.write(str(round(np.mean(num_data),5))+'\n')

std_quads = [gkn_dict,ldfe_dict]
for quad in std_quads:
    rel_diff = np.abs(qr_dict["cad_adj_flux"]-quad["cad_adj_flux"])/qr_dict["cad_adj_flux"]
    rflux_diff = Field(name='rel_diff', axes=ldo_flux.axes, data=rel_diff)
    num_data = map(float,rflux_diff.xs(y=68.58).data.ravel())
    print quad["label"]+"/"+qr_dict["label"]
    print "max diff " + str(max(num_data))
    print "min diff " + str(min(num_data))
    print "avg diff " + str(np.mean(num_data))
    f.write(quad["label"]+"/"+qr_dict["label"]+'\n')
    f.write(str(min(num_data))+' & ')
    f.write(str(round(max(num_data),5))+' & ')
    f.write(str(round(np.mean(num_data),5))+'\n')
    
f.close()

FWCADIS adjoint plotting

In [ ]:
for run in results:
    fullpath = fwc_base_path+run["type"]+"/"+run["name"]+fwc_adj_path
    dnv = load(fullpath)['denovo'].extract()
    axes = dnv.flux.axes[1:]
    flux_data = np.sum(dnv.flux.data, axis=0)
    flux = Field(name='flux', axes=axes, data=flux_data)
    run["fwc_adj_flux"] = flux
    plots = plot(flux.xs(y=68.58), norm=LogNorm(vmin=1e0, vmax=1e8))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    title_name = run["label"]+" "+"%02d" % (run["order"],)
    flux_str = ' FW-CADIS $\phi^{\dagger}\ \\left[\\frac{n}{cm^2\cdot s}\\right]$'
    ax.set_title(title_name + flux_str + ' at $y = 68.58$ cm\n')
    plot_outlines()
    plt.rcParams.update({'axes.titlesize': 9})
    plot_name = 'fwc-adj/flux-'+str(run["name"])+'-slice.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())

FWCADIS adjoint diff plotting

In [ ]:
ldo_flux = ldo_dict["fwc_adj_flux"]
std_quads = [qr_dict,gkn_dict,ldfe_dict]

f = open(plot_dir+'fwc-adj-diff.txt','w')
for quad in std_quads:
    std_flux = quad["fwc_adj_flux"]
    abs_diff = np.abs(ldo_flux - std_flux)
    rel_diff = abs_diff/std_flux
    aflux_diff = Field(name='abs_diff', axes=ldo_flux.axes, data=abs_diff)
    rflux_diff = Field(name='rel_diff', axes=ldo_flux.axes, data=rel_diff)
    plots = plot(aflux_diff.xs(y=68.58), norm=LogNorm(vmin=1e-5, vmax=1e1))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    ax.set_title('LDO/'+quad["label"]+' abs flux diff at $y = 68.58$ cm\n')
    plot_outlines()
    plot_name = 'fwc-adj/flux-diff-abs-'+quad["name"]+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
    #
    plots = plot(rflux_diff.xs(y=68.58),norm=LogNorm(vmin=1e-5, vmax=1e1))
    ax = plots['ax']
    ax.set_xlim(0)
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('z [cm]')
    ax.set_title('LDO/'+quad["label"]+' $\phi^{\dagger}_{\mathrm{diff}}$ at $y = 68.58$ cm\n')
    plot_outlines()
    plt.rcParams.update({'axes.titlesize': 'medium'})
    plot_name = 'fwc-adj/flux-diff-rel-'+quad["name"]+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
    #
    print 'LDO/'+quad["label"]
    num_data = map(float,rflux_diff.xs(y=68.58).data.ravel())
    print "max diff " + str(max(num_data))
    print "min diff " + str(min(num_data))
    print "avg diff " + str(np.mean(num_data))
    f.write('LDO/'+quad["label"]+'\n')
    f.write(str(min(num_data))+' & ')
    f.write(str(round(max(num_data),5))+' & ')
    f.write(str(round(np.mean(num_data),5))+'\n')

std_quads = [gkn_dict,ldfe_dict]
for quad in std_quads:
    rel_diff = np.abs(qr_dict["fwc_adj_flux"]-quad["fwc_adj_flux"])/qr_dict["fwc_adj_flux"]
    rflux_diff = Field(name='rel_diff', axes=ldo_flux.axes, data=rel_diff)
    num_data = map(float,rflux_diff.xs(y=68.58).data.ravel())
    print quad["label"]+"/"+qr_dict["label"]
    print "max diff " + str(max(num_data))
    print "min diff " + str(min(num_data))
    print "avg diff " + str(np.mean(num_data))
    f.write(quad["label"]+"/"+qr_dict["label"]+'\n')
    f.write(str(min(num_data))+' & ')
    f.write(str(round(max(num_data),5))+' & ')
    f.write(str(round(np.mean(num_data),5))+'\n')
    
f.close()

MCNP CADIS

In [8]:
for run in results:
    if run["type"] != "no-ww":
        mctal_file = cad_base_path+run["type"]+"/"+run["name"]+mctal_path
        mc = mctal.load(mctal_file)
        tally_nums = []
        del mc['metadata'] # don't need this
        for tal in mc:
            tally_nums.append(mc[tal].num)
            name = str(mc[tal].num)
            val = float(mc[tal].tfc.data[-1:][0][1])
            err = float(mc[tal].tfc.data[-1:][0][2])
            fom = float(mc[tal].tfc.data[-1:][0][3])
            run[name] = ([mc[tal].num,val,err,fom])
        
mctal_noww = cad_base_path+"no-ww"+mctal_path
mc = mctal.load(mctal_noww)
del mc['metadata'] # don't need this
for tal in mc:
    n = str(mc[tal].num)
    val = float(mc[tal].tfc.data[-1:][0][1])
    err = float(mc[tal].tfc.data[-1:][0][2])
    fom = float(mc[tal].tfc.data[-1:][0][3])
    results.append({"type": "no-ww", 
                    n:[mc[tal].num,val,err,fom]})

qr_tally = sorted([run for run in results \
                   if run["type"] == "qr"], \
                   key=lambda k: k['angles'])
ldo_tally = sorted([run for run in results \
                    if run["type"] == "ldo"], \
                    key=lambda k: k['angles'])
gkn_tally = sorted([run for run in results \
                    if run["type"] == "galerkin"], \
                    key=lambda k: k['angles'])[:2] # only 02 and 04
ldfe_tally = sorted([run for run in results \
                     if run["type"] == "ldfe"], \
                     key=lambda k: k['angles'])[:2] # only 01 and 02

qa = []
for qt in qr_tally:
    qa.append(qt["angles"])
la = []
for lt in ldo_tally:
    la.append(lt["angles"])
ga = []
for gt in gkn_tally:
    ga.append(gt["angles"])
da = []
for dt in ldfe_tally:
    da.append(dt["angles"])
    
det_labels = {14:14,24:12,34:13,44:11,54:9,64:5}
# exp_vals = {14:det14["exp_flux"],24:det12["exp_flux"], \
#             34:det13["exp_flux"],44:det11["exp_flux"], \
#             54: det9["exp_flux"],64: det5["exp_flux"]}

f = open(plot_dir+'cad-det.txt','w')
for t in tally_nums:
    print "Detector Tally #"+str(det_labels[t])
#     f.write("Detector Tally #"+str(det_labels[t])+'\n')
#     f.write("Exp. Flux " + str(exp_vals[t])+'\n')
    qm, qe, qo = np.array([]),np.array([]),np.array([])
    for qt in qr_tally:
        qm = np.append(qm,qt[str(t)][1]) # tally value
        qe = np.append(qe,qt[str(t)][2]) # tally rel. err.
        qo = np.append(qo,qt[str(t)][3]) # FoM
    gm, ge, go = np.array([]),np.array([]),np.array([])
    for gt in gkn_tally:
        gm = np.append(gm,gt[str(t)][1])
        ge = np.append(ge,gt[str(t)][2])
        go = np.append(go,gt[str(t)][3])
    lm, le, lo = np.array([]),np.array([]),np.array([])
    for lt in ldo_tally:
        lm = np.append(lm,lt[str(t)][1])
        le = np.append(le,lt[str(t)][2])
        lo = np.append(lo,lt[str(t)][3])
    dm, de, do = np.array([]),np.array([]),np.array([])
    for dt in ldfe_tally:
        dm = np.append(dm,dt[str(t)][1])
        de = np.append(de,dt[str(t)][2])
        do = np.append(do,dt[str(t)][3])
    avg_fom = np.mean([qr_dict[str(t)][3], gkn_dict[str(t)][3], \
                       ldfe_dict[str(t)][3], ldo_dict[str(t)][3]])
    avg_re  = np.mean([qr_dict[str(t)][2], gkn_dict[str(t)][2], \
                       ldfe_dict[str(t)][2], ldo_dict[str(t)][2]])
    print "QR   FOM " + str(qr_dict[str(t)][3])
    print "Gkn  FOM " + str(gkn_dict[str(t)][3])
    print "LDFE FOM " + str(ldfe_dict[str(t)][3])
    print "LDO  FOM " + str(ldo_dict[str(t)][3])
    print "avg  FOM " + str(avg_fom)
    print "QR   RE " + str(qr_dict[str(t)][2])
    print "Gkn  RE " + str(gkn_dict[str(t)][2])
    print "LDFE RE " + str(ldfe_dict[str(t)][2])
    print "LDO  RE " + str(ldo_dict[str(t)][2])
    print "avg  RE " + str(avg_re)
    f.write("QR   Flux " + str(qr_dict[str(t)][1])+'\t')
    f.write("QR   FOM " + str(qr_dict[str(t)][3])+'\n')
    f.write("Gkn  Flux " + str(gkn_dict[str(t)][1])+'\t')
    f.write("Gkn  FOM " + str(gkn_dict[str(t)][3])+'\n')
    f.write("LDFE Flux " + str(ldfe_dict[str(t)][1])+'\t')
    f.write("LDFE FOM " + str(ldfe_dict[str(t)][3])+'\n')
    f.write("LDO  Flux " + str(ldo_dict[str(t)][1])+'\t')
    f.write("LDO  FOM " + str(ldo_dict[str(t)][3])+'\n')
    for run in results:
        if run["type"] == "no-ww" and str(t) in run:
            nw_tally = run[str(t)][1]
            nw_error = run[str(t)][2]
            nw_fom   = run[str(t)][3]
            print "analog FOM " + str(nw_fom)
            print "analog RE  " + str(nw_error)
            f.write("anlg Flux "+str(nw_tally)+'\t')
            f.write("anlg FOM " + str(nw_fom)+'\n')
    f.write("avg  FOM " + str(avg_fom)+'\n')
    plt.figure()
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    plt.sca(ax)
    plt.errorbar(qa, qm, fmt='o-', yerr=qm*qe, label = 'QR', mec='none')
    plt.errorbar(ga, gm, fmt='s-', yerr=gm*ge, label = 'Galerkin', mec='none')
    plt.errorbar(da, dm, fmt='^-', yerr=dm*de, label = 'LDFE', mec='none')
    plt.errorbar(la, lm, fmt='D-', yerr=lm*le, label = 'LDO', mec='none')
#     plt.axhline(y=nw_tally*(1.0+nw_error), color='k', linestyle='--')
#     plt.axhline(y=nw_tally*(1.0-nw_error), color='k', linestyle='--')
#     plt.axhline(y=nw_tally, color='k', linestyle='-', label = 'No VR')
    plt.title("DLVN Flux Tally at Location of Detector #"+str(det_labels[t]))
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Flux Tally [n/cm$^2$/s]")
    handles, labels = plt.gca().get_legend_handles_labels()
#     order = [1,2,3,4,0]
    order = [0,1,2,3]
    plt.legend([handles[idx] for idx in order], \
               [labels[idx] for idx in order],loc='lower right')
    plot_name = 'mcnp/cadis-tally-'+str(t)+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
    #
    plt.figure()
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    plt.sca(ax)
    plt.plot(qa, qo, 'o-', label = 'QR', mec='none')
    plt.plot(ga, go, 's-', label = 'Galerkin', mec='none')
    plt.plot(da, do, '^-', label = 'LDFE', mec='none')
    plt.plot(la, lo, 'D-', label = 'LDO', mec='none')
#     plt.axhline(y=nw_fom, color='k', linestyle='-', label = 'No VR')
    plt.yscale('log')
    if t == 14:
        plt.ylim(ymin=0.5,ymax=1100)
    plt.title("DLVN FOM for Tally at Location of Detector #"+str(det_labels[t]))
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Reported FOM")
    plt.legend(loc='center right')
    plot_name = 'mcnp/cadis-fom-'+str(t)+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
###
### plotting for MC VR paper
###
    plt.figure()
    plt.subplot(1, 2, 1)
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    ax.ticklabel_format(style='sci',axis='y', scilimits=(0,0))
    plt.sca(ax)
    msize = 4.5
    plt.errorbar(qa, qm, fmt='o-', yerr=qm*qe, label = 'QR', mec='none',markersize=msize)
    plt.errorbar(ga, gm, fmt='s-', yerr=gm*ge, label = 'Galerkin', mec='none',markersize=msize)
    plt.errorbar(da, dm, fmt='^-', yerr=dm*de, label = 'LDFE', mec='none',markersize=msize)
    plt.errorbar(la, lm, fmt='D-', yerr=lm*le, label = 'LDO', mec='none',markersize=msize)
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Flux Tally [n/cm$^2$/s]")
    #
    plt.subplot(1, 2, 2)
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    plt.sca(ax)
    plt.plot(qa, qo, 'o-', label = 'QR', mec='none',markersize=msize)
    plt.plot(ga, go, 's-', label = 'Galerkin', mec='none',markersize=msize)
    plt.plot(da, do, '^-', label = 'LDFE', mec='none',markersize=msize)
    plt.plot(la, lo, 'D-', label = 'LDO', mec='none',markersize=msize)
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Reported FOM")
    plt.yscale('log')
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    plt.legend(loc='lower right')
    fig = plt.gcf()
    fig.suptitle("DLVN Benchmark Flux Tallies and FOM Values at Detector #"+str(det_labels[t]))
    plot_name = 'mcnp/cadis-'+str(t)+'.eps'
    plt.savefig(plot_dir+plot_name,format='eps')
    plt.close(plt.gcf())
f.close()

Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-cadis/galerkin/gkn04/output/mctal
INFO: Loading mctal from problem name 'dlvn'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-cadis/galerkin/gkn08/output/mctal
INFO: Loading mctal from problem name 'dlvn'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-cadis/galerkin/gkn06/output/mctal
INFO: Loading mctal from problem name 'dlvn'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-cadis/galerkin/gkn02/output/mctal
INFO: Loading mctal from problem name 'dlvn'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-cadis/ldfe/ldfe01/output/mctal
INFO: Loading mctal from pro

Detector Tally #5
QR   FOM 8.12501
Gkn  FOM 9.49132
LDFE FOM 6.79831
LDO  FOM 8.40981
avg  FOM 8.2061125
QR   RE 0.00191423
Gkn  RE 0.00161288
LDFE RE 0.00207173
LDO  RE 0.00186168
avg  RE 0.00186513
analog FOM 0.149122
analog RE  0.0069224
analog FOM 0.149122
analog RE  0.0069224
Detector Tally #14
QR   FOM 788.016
Gkn  FOM 143.411
LDFE FOM 879.516
LDO  FOM 537.21
avg  FOM 587.03825
QR   RE 0.000194374
Gkn  RE 0.000414929
LDFE RE 0.000182143
LDO  RE 0.000232931
avg  RE 0.00025609425
analog FOM 0.763622
analog RE  0.00305906
analog FOM 0.763622
analog RE  0.00305906
Detector Tally #11
QR   FOM 0.432798
Gkn  FOM 2.75436
LDFE FOM 0.557676
LDO  FOM 3.42302
avg  FOM 1.7919635
QR   RE 0.00829397
Gkn  RE 0.00299402
LDFE RE 0.00723341
LDO  RE 0.00291806
avg  RE 0.005359865
analog FOM 14.5549
analog RE  0.000700685
analog FOM 14.5549
analog RE  0.000700685
Detector Tally #9
QR   FOM 209.205
Gkn  FOM 189.644
LDFE FOM 214.514
LDO  FOM 205.67
avg  FOM 204.75825
QR   RE 0.000377241
Gkn  RE 0.00036

MCNP FWCADIS

In [19]:
for run in results:
    if run["type"] != "no-ww":
        mctal_file = fwc_base_path+run["type"]+"/"+run["name"]+mctal_path
        mc = mctal.load(mctal_file)
        del mc['metadata'] # don't need this
        for tal in mc:
            name = str(mc[tal].num)
            val = float(mc[tal].tfc.data[-1:][0][1])
            err = float(mc[tal].tfc.data[-1:][0][2])
            fom = float(mc[tal].tfc.data[-1:][0][3])
            run[name] = ([mc[tal].num,val,err,fom])

f = open(plot_dir+'fwc-det.txt','w')
for t in tally_nums:
#     print "Detector Tally #"+str(det_labels[t])
    f.write("Detector Tally #"+str(det_labels[t])+'\n')
    qm, qe, qo = np.array([]),np.array([]),np.array([])
    for qt in qr_tally:
        qm = np.append(qm,qt[str(t)][1]) # tally value
        qe = np.append(qe,qt[str(t)][2]) # tally rel. err.
        qo = np.append(qo,qt[str(t)][3]) # FoM
    gm, ge, go = np.array([]),np.array([]),np.array([])
    for gt in gkn_tally:
        gm = np.append(gm,gt[str(t)][1])
        ge = np.append(ge,gt[str(t)][2])
        go = np.append(go,gt[str(t)][3])
    lm, le, lo = np.array([]),np.array([]),np.array([])
    for lt in ldo_tally:
        lm = np.append(lm,lt[str(t)][1])
        le = np.append(le,lt[str(t)][2])
        lo = np.append(lo,lt[str(t)][3])
    dm, de, do = np.array([]),np.array([]),np.array([])
    for dt in ldfe_tally:
        dm = np.append(dm,dt[str(t)][1])
        de = np.append(de,dt[str(t)][2])
        do = np.append(do,dt[str(t)][3])
    avg_fom = np.mean([qr_dict[str(t)][3], gkn_dict[str(t)][3], \
                       ldfe_dict[str(t)][3], ldo_dict[str(t)][3]])
    avg_re  = np.mean([qr_dict[str(t)][2], gkn_dict[str(t)][2], \
                       ldfe_dict[str(t)][2], ldo_dict[str(t)][2]])
#     print "QR   FOM " + str(qr_dict[str(t)][3])
#     print "Gkn  FOM " + str(gkn_dict[str(t)][3])
#     print "LDFE FOM " + str(ldfe_dict[str(t)][3])
#     print "LDO  FOM " + str(ldo_dict[str(t)][3])
#     print "avg  FOM " + str(avg_fom)
#     print "QR   RE " + str(qr_dict[str(t)][2])
#     print "Gkn  RE " + str(gkn_dict[str(t)][2])
#     print "LDFE RE " + str(ldfe_dict[str(t)][2])
#     print "LDO  RE " + str(ldo_dict[str(t)][2])
#     print "avg  RE " + str(avg_re)
    f.write("QR   Flux " + str(qr_dict[str(t)][1])+'\t')
    f.write("QR   Err " + str(qr_dict[str(t)][1]*qr_dict[str(t)][2])+'\t')
    f.write("QR   FOM " + str(qr_dict[str(t)][3])+'\n')
    f.write("Gkn  Flux " + str(gkn_dict[str(t)][1])+'\t')
    f.write("Gkn  Err " + str(gkn_dict[str(t)][1]*gkn_dict[str(t)][2])+'\t')
    f.write("Gkn  FOM " + str(gkn_dict[str(t)][3])+'\n')
    f.write("LDFE Flux " + str(ldfe_dict[str(t)][1])+'\t')
    f.write("LDFE Err " + str(ldfe_dict[str(t)][1]*ldfe_dict[str(t)][2])+'\t')
    f.write("LDFE FOM " + str(ldfe_dict[str(t)][3])+'\n')
    f.write("LDO  Flux " + str(ldo_dict[str(t)][1])+'\t')
    f.write("LDO  Err " + str(ldo_dict[str(t)][1]*ldo_dict[str(t)][2])+'\t')
    f.write("LDO  FOM " + str(ldo_dict[str(t)][3])+'\n')
    for run in results:
        if run["type"] == "no-ww" and str(t) in run:
            nw_tally = run[str(t)][1]
            nw_error = run[str(t)][2]
            nw_fom   = run[str(t)][3]
#             print "analog FOM " + str(nw_fom)
#             print "analog RE  " + str(nw_error)
            f.write("anlg Flux "+str(nw_tally)+'\t')
            f.write("anlg Err  "+str(nw_tally*nw_error)+'\t')
            f.write("anlg FOM " + str(nw_fom)+'\n')
    f.write("avg  FOM " + str(avg_fom)+'\n')
    plt.figure()
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    plt.sca(ax)
    plt.errorbar(qa, qm, fmt='.-', yerr=qm*qe, label = 'QR')
    plt.errorbar(ga, gm, fmt='.-', yerr=gm*ge, label = 'Galerkin')
    plt.errorbar(da, dm, fmt='.-', yerr=dm*de, label = 'LDFE')
    plt.errorbar(la, lm, fmt='.-', yerr=lm*le, label = 'LDO')
#     plt.axhline(y=nw_tally, color='k', linestyle='-', label='No VR')
#     plt.axhline(y=nw_tally*(1.0+nw_error), color='k', linestyle='--')
#     plt.axhline(y=nw_tally*(1.0-nw_error), color='k', linestyle='--')
    plt.title("DLVN MCNP Tally at Location of Detector #"+str(det_labels[t]))
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("MCNP Tally [n/cm$^2$/s]")
    handles, labels = plt.gca().get_legend_handles_labels()
#     order = [1,2,3,4,0]
    order = [0,1,2,3]
    if t == 14:
        legloc = 'upper right'
    else:
        legloc = 'lower right'
    plt.legend([handles[idx] for idx in order], \
               [labels[idx] for idx in order],loc=legloc)
    plot_name = 'mcnp/fwc-tally-'+str(det_labels[t])+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight')
    plt.close(plt.gcf())
    #
    plt.figure()
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    plt.sca(ax)
    plt.plot(qa, qo, 'o-', label = 'QR')
    plt.plot(ga, go, 'o-', label = 'Galerkin')
    plt.plot(da, do, 'o-', label = 'LDFE')
    plt.plot(la, lo, 'o-', label = 'LDO')
#     plt.axhline(y=nw_fom, color='k', linestyle='-', label='No VR')
    plt.title("DLVN FOM for Tally at Location of Detector #"+str(det_labels[t]))
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Reported FOM")
    plt.yscale('log')
    if t in [14,34,54]:
        plt.legend(loc='center right')
    else:
        plt.legend(loc='lower right')
    if t != 24:
        plt.ylim(ymin=0.2)
    else:
        plt.ylim(ymin=2,ymax=800)
    plot_name = 'mcnp/fwc-fom-'+str(det_labels[t])+'.eps'
    plt.savefig(plot_dir+plot_name,bbox_inches='tight',format='eps')
    plt.close(plt.gcf())
f.close()

quads = [qr_dict,gkn_dict,ldfe_dict,ldo_dict]
for t in tally_nums:
    qm, qe, qo = np.array([]),np.array([]),np.array([])
    for qt in qr_tally:
        qm = np.append(qm,qt[str(t)][1]) # tally value
        qe = np.append(qe,qt[str(t)][2]) # tally rel. err.
        qo = np.append(qo,qt[str(t)][3]) # FoM
    gm, ge, go = np.array([]),np.array([]),np.array([])
    for gt in gkn_tally:
        gm = np.append(gm,gt[str(t)][1])
        ge = np.append(ge,gt[str(t)][2])
        go = np.append(go,gt[str(t)][3])
    lm, le, lo = np.array([]),np.array([]),np.array([])
    for lt in ldo_tally:
        lm = np.append(lm,lt[str(t)][1])
        le = np.append(le,lt[str(t)][2])
        lo = np.append(lo,lt[str(t)][3])
    dm, de, do = np.array([]),np.array([]),np.array([])
    for dt in ldfe_tally:
        dm = np.append(dm,dt[str(t)][1])
        de = np.append(de,dt[str(t)][2])
        do = np.append(do,dt[str(t)][3])
#     for run in results:
#         if run["type"] == "no-ww" and str(t) in run:
#             nw_tally = run[str(t)][1]
#             nw_error = run[str(t)][2]
#             nw_fom   = run[str(t)][3]
    plt.figure()
    plt.subplot(1, 2, 1)
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    ax.ticklabel_format(style='sci',axis='y', scilimits=(0,0))
    plt.sca(ax)
    msize = 4.5
    plt.errorbar(qa, qm, fmt='o-', yerr=qm*qe, label = 'QR', mec='none',markersize=msize)
    plt.errorbar(ga, gm, fmt='s-', yerr=gm*ge, label = 'Galerkin', mec='none',markersize=msize)
    plt.errorbar(da, dm, fmt='^-', yerr=dm*de, label = 'LDFE', mec='none',markersize=msize)
    plt.errorbar(la, lm, fmt='D-', yerr=lm*le, label = 'LDO', mec='none',markersize=msize)
#     plt.axhline(y=nw_tally, color='k', linestyle='-', label='No VR')
#     plt.axhline(y=nw_tally*(1.0+nw_error), color='k', linestyle='--')
#     plt.axhline(y=nw_tally*(1.0-nw_error), color='k', linestyle='--')
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Flux Tally [n/cm$^2$/s]")
    #
    plt.subplot(1, 2, 2)
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    plt.sca(ax)
    plt.plot(qa, qo, 'o-', label = 'QR', mec='none',markersize=msize)
    plt.plot(ga, go, 's-', label = 'Galerkin', mec='none',markersize=msize)
    plt.plot(da, do, '^-', label = 'LDFE', mec='none',markersize=msize)
    plt.plot(la, lo, 'D-', label = 'LDO', mec='none',markersize=msize)
#     plt.axhline(y=nw_fom, color='k', linestyle='-', label='No VR')
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Reported FOM")
    plt.yscale('log')
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    if det_labels[t] in [9]:
        plt.legend(loc='center right')
    elif det_labels[t] in [11]:
        plt.legend(loc='lower center')
    elif det_labels[t] in [12]:
        plt.legend(loc='lower left')
    else:
        plt.legend(loc='lower right')
    fig = plt.gcf()
    fig.suptitle("DLVN Benchmark Flux Tallies and FOM Values at Detector #"+str(det_labels[t]))
    plot_name = 'mcnp/fwc-'+str(det_labels[t])+'.eps'
    plt.savefig(plot_dir+plot_name,format='eps')
    plt.close(plt.gcf())

Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-fwcadis/galerkin/gkn04/output/mctal
INFO: Loading mctal from problem name 'dlvn'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-fwcadis/galerkin/gkn08/output/mctal
INFO: Loading mctal from problem name 'dlvn'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-fwcadis/galerkin/gkn06/output/mctal
INFO: Loading mctal from problem name 'dlvn'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-fwcadis/galerkin/gkn02/output/mctal
INFO: Loading mctal from problem name 'dlvn'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from /home/krowland/ldo-results/dlvn-fwcadis/ldfe/ldfe01/output/mctal
INFO: Loading mcta

In [15]:
for t in tally_nums:
    qa = qr_dict["angles"]
    qm = qr_dict[str(t)][1]
    qe = qr_dict[str(t)][2]
    ga = gkn_dict["angles"]
    gm = gkn_dict[str(t)][1]
    ge = gkn_dict[str(t)][2]
    da = ldfe_dict["angles"]
    dm = ldfe_dict[str(t)][1]
    de = ldfe_dict[str(t)][2]
    la = ldo_dict["angles"]
    lm = ldo_dict[str(t)][1]
    le = ldo_dict[str(t)][2]
    for run in results:
        if run["type"] == "no-ww" and str(t) in run:
            nw_tally = run[str(t)][1]
            nw_error = run[str(t)][2]
            nw_fom   = run[str(t)][3]
    plt.figure()
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    plt.sca(ax)
    plt.errorbar(qa, qm, fmt='o-', yerr=qm*qe, label = 'QR', mec='none')
    plt.errorbar(ga, gm, fmt='s-', yerr=gm*ge, label = 'Galerkin', mec='none')
    plt.errorbar(da, dm, fmt='^-', yerr=dm*de, label = 'LDFE', mec='none')
    plt.errorbar(la, lm, fmt='D-', yerr=lm*le, label = 'LDO', mec='none')
#     plt.axhline(y=nw_tally, color='k', linestyle='-', label='No VR')
#     plt.axhline(y=nw_tally*(1.0+nw_error), color='k', linestyle='--')
#     plt.axhline(y=nw_tally*(1.0-nw_error), color='k', linestyle='--')
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Flux Tally [n/cm$^2$/s]")
    if t == 34:
        legloc = 'upper right'
    else:
        legloc = 'lower right'
    handles, labels = plt.gca().get_legend_handles_labels()
#     order = [1,2,3,4,0]
    order = [0,1,2,3]
    plt.legend([handles[idx] for idx in order], \
               [labels[idx] for idx in order],loc=legloc)
    fig = plt.gcf()
    fig.suptitle("DLVN Benchmark Flux Tallies at Detector #"+str(det_labels[t]))
    plot_name = 'mcnp/fwc-rep-'+str(det_labels[t])+'.eps'
    plt.savefig(plot_dir+plot_name)
    plt.close(plt.gcf())